In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from pytz import timezone

In [2]:
df = pd.read_json('/Users/Karolina/ProjectsDataScience/data_science_environment/data/MyDataSpotify/StreamingHistory4.json')

In [3]:
df.head()

,endTime,artistName,trackName,msPlayed
0,2021-10-02 20:31,Surf Mesa,Carried Away (Love To Love) (with Madison Beer),182629
1,2021-10-03 07:33,Mr.Kitty,After Dark,116403
2,2021-10-03 07:33,Khalid,Better,1493
3,2021-10-03 07:37,Khalid,Better,228583
4,2021-10-03 07:41,Khalid,Better,229275


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7735 entries, 0 to 7734
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     7735 non-null   object
 1   artistName  7735 non-null   object
 2   trackName   7735 non-null   object
 3   msPlayed    7735 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 241.8+ KB


In [5]:
df.endTime = pd.to_datetime(df.endTime, infer_datetime_format=True) 

In [6]:
#Set the current time to US Pacific timezone
df['endTimeTz']= df.endTime.dt.tz_localize(tz='UTC')

#convert US Pacific to Europe/Berlin Timezone
df['endTimeTzEU']=df.endTimeTz.dt.tz_convert(tz='Europe/Berlin')

#create year-week column (Week starts on Monday)
df['yearWeek']=df.endTimeTzEU.dt.strftime("%Y-%W")



In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7735 entries, 0 to 7734
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype                        
---  ------       --------------  -----                        
 0   endTime      7735 non-null   datetime64[ns]               
 1   artistName   7735 non-null   object                       
 2   trackName    7735 non-null   object                       
 3   msPlayed     7735 non-null   int64                        
 4   endTimeTz    7735 non-null   datetime64[ns, UTC]          
 5   endTimeTzEU  7735 non-null   datetime64[ns, Europe/Berlin]
 6   yearWeek     7735 non-null   object                       
dtypes: datetime64[ns, Europe/Berlin](1), datetime64[ns, UTC](1), datetime64[ns](1), int64(1), object(3)
memory usage: 423.1+ KB


In [8]:
df.groupby(['yearWeek','artistName']).count()

endTime  trackName  msPlayed  endTimeTz  \
yearWeek artistName                                                          
2021-39  Billie Eilish                   2          2         2          2   
         CKay                           18         18        18         18   
         Drake                           3          3         3          3   
         Kehlani                         9          9         9          9   
         Khalid                          4          4         4          4   
...                                    ...        ...       ...        ...   
2021-46  Two Feet                        2          2         2          2   
         Tyga                            1          1         1          1   
         Unknown Mortal Orchestra        2          2         2          2   
         Young the Giant                 1          1         1          1   
         dvsn                            1          1         1          1   

                                   endTimeTzEU  
yearWeek artistName                             
2021-39  Billie Eilish                       2  
         CKay                               18  
         Drake                               3  
         Kehlani                             9  
         Khalid                              4  
...                                        ...  
2021-46  Two Feet                            2  
         Tyga                                1  
         Unknown Mortal Orchestra            2  
         Young the Giant                     1  
         dvsn                                1  

[1629 rows x 5 columns]

In [9]:
weekly = df.groupby([pd.Grouper(key='endTime', freq='W'),'artistName']).size()

In [10]:
weekly5 = weekly.groupby(level=0, group_keys=False).nlargest(5)

weekly5_u = weekly5.unstack()
weekly5_u.fillna(0, inplace=True)

In [11]:
import bar_chart_race as bcr


In [18]:
%%time
bcr.bar_chart_race(df=weekly5_u, n_bars=5, title='bla', period_length=3000, cmap='prism', filename='t')

/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


CPU times: user 4.52 s, sys: 1.08 s, total: 5.6 s
Wall time: 6.49 s


In [19]:
help(bcr.bar_chart_race)

Help on function bar_chart_race in module bar_chart_race._make_chart:

bar_chart_race(df, filename=None, orientation='h', sort='desc', n_bars=None, fixed_order=False, fixed_max=False, steps_per_period=10, period_length=500, interpolate_period=False, label_bars=True, bar_size=0.95, period_label=True, period_fmt=None, period_summary_func=None, perpendicular_bar_func=None, figsize=(6, 3.5), cmap=None, title=None, title_size=None, bar_label_size=7, tick_label_size=7, shared_fontdict=None, scale='linear', writer=None, fig=None, dpi=144, bar_kwargs=None, filter_column_colors=False)
    Create an animated bar chart race using matplotlib. Data must be in 
    'wide' format where each row represents a single time period and each 
    column represents a distinct category. Optionally, the index can label 
    the time period.
    
    Bar height and location change linearly from one time period to the next.
    
    If no `filename` is given, an HTML string is returned, otherwise the 
    animat